# Projet de Python

### ERRAKKAS Yassine , INE 1 Data

#### l'idée du projet

Ce projet est la première partie d'un grand projet, ce grand projet concerne un robot de recommandation, il vous suffit de lui donner un article que vous voulez acheter en ligne, et il vous donnera le lien du meilleur article.
Pour cette première partie, nous nous concentrons sur le scraping de données, nous commencerons par ebay, aliexpress et amazon pour ce prototype. par la suite, nous ajouterons d'autres sites web.

#### L'implémentation avec python

les bibliothèques que nous utilisons

In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv

appeler le navigateur pour obtenir les sites web

In [2]:
driver = webdriver.Chrome()

Cette fonction prend en paramètre un Xpath d'éléments et retourne le texte dans ces éléments si on donne 0 dans le deuxieme paramètre et les liens si on donne 1.


In [42]:
def element_xpath_to_text(xpath, option):
    element_list = driver.find_elements(
        By.XPATH, xpath)
    elements = []
    if option == 0:
        for element in element_list:
            elements.append(element.text)
    elif option == 1:
        for element in element_list:
            elements.append(element.get_attribute('href'))
    return elements


Sur ebay, on trouve parfois dans un élément la phrase "Top rated seller" et le nom du vendeur. C'est pourquoi nous avons écrit cette fonction pour supprimer cette phrase et ne garder que le nom du vendeur.

In [43]:
def saller_names_filter(nonFiltredNames):
    filtrednames = []
    for i in nonFiltredNames:
        if i == 'Top Rated Seller':
            continue
        else :
            filtrednames.append(i.split()[0])
    return filtrednames


Ces trois fonctions renvoient un dictionnaire qui contient les titres, les prix, les vendeurs et les liens de tous les articles de la page. la première est pour ebay, la deuxième pour aliexpress et la troisième pour amazon.

In [44]:
def ebayScraper(input):
    
    ebay_title_xpath = '//*[@id="srp-river-results"]/ul/li/div/div[2]/a/div/span'
    ebay_itemLink_xpath = '//*[@id="srp-river-results"]/ul/li/div/div[2]/a'
    ebay_price_xpath = '//*[@id="srp-river-results"]/ul/li/div/div[2]/div/div[1]/span'
    ebay_seller_xpath = '//*[@id = "srp-river-results"]/ul/li/div/div[2]/div/span/span/span'

    driver.get('https://www.ebay.com/')
    # search bar
    driver.find_element(By.XPATH, '//*[@id="gh-ac"]').send_keys(input)
    # search button
    driver.find_element(By.XPATH, '//*[@id="gh-btn"]').click()
    
    titles = element_xpath_to_text(ebay_title_xpath, 0)
    links = element_xpath_to_text(ebay_itemLink_xpath, 1)
    prices = element_xpath_to_text(ebay_price_xpath, 0)
    sellers = saller_names_filter(element_xpath_to_text(ebay_seller_xpath, 0))
    return {'titles': titles, 'prices': prices, 'sellers': sellers, 'links': links}

In [45]:
def aliexpressScraper(input):
    
    ali_title_xpath = '// *[@id="root"]/div/div/div[2]/div/div[2]/div[3]/a/div[2]/div[4]/h1'
    ali_itemLink_xpath = '//*[@id="root"]/div/div/div[2]/div/div[2]/div[3]/a'
    ali_price_xpath = '//*[@id="root"]/div/div/div[2]/div/div[2]/div[3]/a/div[2]/div[1]/div[1]'
    ali_seller_xpath = '//*[@id="root"]/div/div/div[2]/div/div[2]/div[3]/a/div[2]/span/a'

    driver.get('https://www.aliexpress.com/')
    # search bar
    driver.find_element(By.XPATH, '// *[@id="search-key"]').send_keys(input)
    # search button 
    driver.find_element(By.XPATH, '//*[@id = "form-searchbar"]/div[1]/input').click()
    # for more content
    driver.execute_script('window.scrollTo(e, document.body.scrollHeight);')

    titles = element_xpath_to_text(ali_title_xpath, 0)
    prices = element_xpath_to_text(ali_price_xpath, 0)
    sellers = element_xpath_to_text(ali_seller_xpath, 0)
    links = element_xpath_to_text(ali_itemLink_xpath, 1)
    return {'titles': titles, 'prices': prices, 'sellers': sellers, 'links': links}


In [126]:
def AmazonScaper(input):
    driver.get('https://www.amazon.com/')
    driver.find_element(
        By.XPATH, '//*[@id="twotabsearchtextbox"]').send_keys(input)
    driver.find_element(
        By.XPATH, '//*[@id="nav-search-submit-button"]').click()
    html = driver.page_source
    source = BeautifulSoup(html, "lxml")
    itemsdiv = source.find_all(
        'div', {'data-asin': True, 'data-component-type': 's-search-result'})
    titles = []
    links = []
    prices = []
    sellers = []
    for itemdiv in itemsdiv:
        try:
            h2 = itemdiv.h2
        except:
            continue
        else:
            titles.append(h2.text)
            links.append('https://www.amazon.com'+h2.a.get('href'))
            prices.append(itemdiv.find('span', class_='a-price-symbol').text+itemdiv.find('span', class_='a-price-whole').text +
                                itemdiv.find('span', class_='a-price-fraction').text)
            sellers.append('Null')
    return {'titles' : titles,'prices':prices,'sellers':sellers,'links':links}


cette fonction permet d'écrire un fichier csv qui contient toutes les données que nous récupérons, elle prend un dictionnaire de données et le nom du fichier csv 

In [124]:
def Write_csv_file(data,filename):
    with open(filename+'.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(data.keys())
        for i in range(len(list(data.values())[0])):
            writer.writerow(val[i] for val in list(data.values()))


un exemple


In [122]:
data = ebayScraper('galaxy s10 case')
Write_csv_file(data,'ebay_data')